In [4]:
import requests
import random
import json

In [5]:
base_url = "http://localhost:8081"
keycloak_url = "http://keycloak:8080"

In [6]:
resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"

ConnectionError: HTTPConnectionPool(host='localhost', port=8081): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd10431a620>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:

resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

In [ ]:
swaggerJson = resp.json()
swaggerJson["paths"].keys()

dict_keys(['/api/v1/Institute/{entityId}', '/api/docs/Institute.json', '/api/v1/Institute/sign', '/api/v1/Institute/search', '/api/v1/Institute/invite', '/api/v1/Institute', '/api/v1/Institute/{entityId}/{property}/{propertyId}', '/api/v1/Institute/{entityId}/{property}', '/api/v1/Institute/{entityId}/{property}/{propertyId}/send', '/api/v1/Schema/{entityId}', '/api/docs/Schema.json', '/api/v1/Schema/sign', '/api/v1/Schema/search', '/api/v1/Schema/invite', '/api/v1/Schema', '/api/v1/Schema/{entityId}/{property}/{propertyId}', '/api/v1/Schema/{entityId}/{property}', '/api/v1/Schema/{entityId}/{property}/{propertyId}/send', '/api/v1/Teacher/{entityId}', '/api/docs/Teacher.json', '/api/v1/Teacher/sign', '/api/v1/Teacher/search', '/api/v1/Teacher/invite', '/api/v1/Teacher', '/api/v1/Teacher/{entityId}/{property}/{propertyId}', '/api/v1/Teacher/{entityId}/{property}', '/api/v1/Teacher/{entityId}/{property}/{propertyId}/send', '/api/v1/BaseAttestationField/{entityId}', '/api/docs/BaseAttesta

In [ ]:
jsonUrl = [f for f in swaggerJson["paths"].keys() if ".json" in f][0]
jsonUrl

'/api/docs/Institute.json'

In [ ]:
resp = requests.get("%s%s"%(base_url, jsonUrl))
assert resp.status_code == 200

In [ ]:
resp.json()

{'Institute': {'$id': '#/properties/Institute',
  'type': 'object',
  'title': 'The Institute Schema',
  'required': [],
  'properties': {'instituteName': {'$id': '#/properties/instituteName',
    'type': 'string',
    'title': 'Institute Name'},
   'address': {'type': 'object',
    'required': [],
    'properties': {'plot': {'type': 'string', 'title': 'Plot'},
     'street': {'type': 'string', 'title': 'Street'},
     'landmark': {'type': 'string', 'title': 'Landmark'},
     'locality': {'type': 'string', 'title': 'Locality'},
     'state': {'type': 'string',
      'title': 'State',
      'enum': ['Andaman and Nicobar Islands',
       'Andhra Pradesh',
       'Arunachal Pradesh',
       'Assam',
       'Bihar',
       'Chandigarh',
       'Chhattisgarh',
       'Dadra and Nagar Haveli',
       'Daman and Diu',
       'Delhi',
       'Goa',
       'Gujarat',
       'Haryana',
       'Himachal Pradesh',
       'Jammu and Kashmir',
       'Jharkhand',
       'Karnataka',
       'Kerala',

In [ ]:
[f for f in swaggerJson["paths"].keys() if "/invite" in f]

['/api/v1/Institute/invite',
 '/api/v1/Schema/invite',
 '/api/v1/Teacher/invite',
 '/api/v1/BaseAttestationField/invite',
 '/api/v1/Student/invite',
 '/api/v1/Common/invite']

In [ ]:
inviteUrl = [f for f in swaggerJson["paths"].keys() if "/invite" in f][2]
inviteUrl

'/api/v1/Teacher/invite'

In [ ]:
def entity_for(userId):
    return  {
    "name" : "Harshil",
    "phoneNumber" : "72390",
    "email" : "haj@1234.com",
    "subject" : "Electrical Kernel",
    "school" : "NIT Surat",
      "identityDetails":{"type":"id","value":userId}, 
      "contactDetails":{ "mobile":"%s"%userId,"email":"%s@example.com"%userId,
                       "address":{"street":"abc","state":"Karnataka","pincode":"560001"}}
     }

In [ ]:
userId=12
json.dumps(entity_for(userId))

'{"name": "Harshil", "phoneNumber": "72390", "email": "haj@1234.com", "subject": "Electrical Kernel", "school": "NIT Surat", "identityDetails": {"type": "id", "value": 12}, "contactDetails": {"mobile": "12", "email": "12@example.com", "address": {"street": "abc", "state": "Karnataka", "pincode": "560001"}}}'

# invite

In [ ]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, inviteUrl), json=entity_for(userId)
)
print(userId)
print()
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Teacher"==entity_name
idx = resp.json()["result"][entity_name]["osid"]

50693540130

b'{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1656617023281,"params":{"resmsgid":"","msgid":"7039c285-9aa9-40d4-9864-d158966c28a8","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Teacher":{"osid":"1-bf40f035-5293-4994-baf2-abb7c01880f9"}}}'
{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1656617023281, 'params': {'resmsgid': '', 'msgid': '7039c285-9aa9-40d4-9864-d158966c28a8', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Teacher': {'osid': '1-bf40f035-5293-4994-baf2-abb7c01880f9'}}}


In [ ]:
idx

'1-bf40f035-5293-4994-baf2-abb7c01880f9'

In [ ]:
userId

'50693540130'

In [ ]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}

data = {
  'client_id': 'registry-frontend',
  'username': userId,
  'password': 'abcd@123',
  'grant_type': 'password'
}

response = requests.post('%s/auth/realms/sunbird-rc/protocol/openid-connect/token'%(keycloak_url), headers=headers, data=data)
print(response.json())
print(response.status_code)
token = response.json()["access_token"]


{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJjQVJSY3JFdWZ5TnNUR1laZlBmNUZyVGtFVzgwVHpMaC11aTc2MEZCM3o4In0.eyJleHAiOjE2NTY2MTc2MjMsImlhdCI6MTY1NjYxNzAyMywianRpIjoiNTBhZWI5YzItYjFmZi00ZDViLTllM2MtOTVkZmRlMTZhZmQzIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI0Njk1ODNiOC1hNDg2LTQ0MjctYjM4My0wYWYxYzRlZWNiODYiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI2MjMxYjY0Zi00NjI3LTQxMjItYTUyNS0yNThlZDk5ZmNlMmUiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidm

In [ ]:
token


'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJjQVJSY3JFdWZ5TnNUR1laZlBmNUZyVGtFVzgwVHpMaC11aTc2MEZCM3o4In0.eyJleHAiOjE2NTY2MTc2MjMsImlhdCI6MTY1NjYxNzAyMywianRpIjoiNTBhZWI5YzItYjFmZi00ZDViLTllM2MtOTVkZmRlMTZhZmQzIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI0Njk1ODNiOC1hNDg2LTQ0MjctYjM4My0wYWYxYzRlZWNiODYiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI2MjMxYjY0Zi00NjI3LTQxMjItYTUyNS0yNThlZDk5ZmNlMmUiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl1

In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

resp = requests.get("%s/api/docs/swagger.json"%base_url, headers=headers)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%(entity_name, idx), headers=headers)
print(response.status_code)
obj = response.json()
response.status_code, response.json()

200


(200,
 {'osUpdatedAt': '2022-06-30T19:23:43.355Z',
  'osUpdatedBy': '',
  'subject': 'Electrical Kernel',
  'osid': '1-bf40f035-5293-4994-baf2-abb7c01880f9',
  'osOwner': ['469583b8-a486-4427-b383-0af1c4eecb86'],
  'phoneNumber': '72390',
  'osCreatedAt': '2022-06-30T19:23:43.355Z',
  'school': 'NIT Surat',
  'contactDetails': {'osid': '1-4e031217-9dc3-41b6-b4c8-11718ddf9dfe',
   'osUpdatedAt': '2022-06-30T19:23:43.355Z',
   'osCreatedAt': '2022-06-30T19:23:43.355Z',
   'address': {'osid': '1-a2f2170c-72b5-4e1a-bcb5-2b5122a67595',
    'osUpdatedAt': '2022-06-30T19:23:43.355Z',
    'pincode': '560001',
    'osCreatedAt': '2022-06-30T19:23:43.355Z',
    'osUpdatedBy': '',
    'street': 'abc',
    'osCreatedBy': '',
    'state': 'Karnataka'},
   'osUpdatedBy': '',
   'mobile': '50693540130',
   'osCreatedBy': '',
   'email': '50693540130@example.com'},
  'name': 'Harshil',
  'osCreatedBy': '',
  'identityDetails': {'osid': '1-01f845e4-19f4-47a8-a46d-d007ca8b0838',
   'osUpdatedAt': '2022-

## update name and experience

In [ ]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

data = '{"contactDetails":{"name": "test name %s"}, "experience":[{"institute":"Some other school"}]}'%userId
print(data)
response = requests.put('http://localhost:8081/api/v1/Teacher/%s'%idx, headers=headers, data=data)

{"contactDetails":{"name": "test name 50693540130"}, "experience":[{"institute":"Some other school"}]}


In [ ]:
response.status_code, response.json()

(200,
 {'id': 'sunbird-rc.registry.update',
  'ver': '1.0',
  'ets': 1656617023879,
  'params': {'resmsgid': '',
   'msgid': 'a8fa32b0-3919-41d7-9bf1-8ea73109ef93',
   'err': '',
   'status': 'SUCCESSFUL',
   'errmsg': ''},
  'responseCode': 'OK'})

In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%(entity_name, idx), headers=headers)
assert response.status_code == 200
print(response.json())
assert response.json()["contactDetails"]["name"] == "test name %s"%userId
response.json()

{'osUpdatedAt': '2022-06-30T19:23:43.925Z', 'osUpdatedBy': '469583b8-a486-4427-b383-0af1c4eecb86', 'subject': 'Electrical Kernel', 'osid': '1-bf40f035-5293-4994-baf2-abb7c01880f9', 'osOwner': ['469583b8-a486-4427-b383-0af1c4eecb86'], 'phoneNumber': '72390', 'osCreatedAt': '2022-06-30T19:23:43.355Z', 'school': 'NIT Surat', 'contactDetails': {'osid': '1-989bfb8e-cce3-463d-9343-cc71d16796b1', 'osUpdatedAt': '2022-06-30T19:23:43.925Z', 'osCreatedAt': '2022-06-30T19:23:43.355Z', 'osUpdatedBy': '469583b8-a486-4427-b383-0af1c4eecb86', 'mobile': '50693540130', 'osCreatedBy': '', 'name': 'test name 50693540130', 'email': '50693540130@example.com'}, 'name': 'Harshil', 'osCreatedBy': '', 'experience': [{'osUpdatedAt': '2022-06-30T19:23:43.925Z', 'osUpdatedBy': '469583b8-a486-4427-b383-0af1c4eecb86', 'institute': 'Some other school', 'osid': '1-29ec18c1-a464-466b-a31e-5c96924f77ba'}], 'identityDetails': {'osid': '1-01f845e4-19f4-47a8-a46d-d007ca8b0838', 'osUpdatedAt': '2022-06-30T19:23:43.925Z', '

{'osUpdatedAt': '2022-06-30T19:23:43.925Z',
 'osUpdatedBy': '469583b8-a486-4427-b383-0af1c4eecb86',
 'subject': 'Electrical Kernel',
 'osid': '1-bf40f035-5293-4994-baf2-abb7c01880f9',
 'osOwner': ['469583b8-a486-4427-b383-0af1c4eecb86'],
 'phoneNumber': '72390',
 'osCreatedAt': '2022-06-30T19:23:43.355Z',
 'school': 'NIT Surat',
 'contactDetails': {'osid': '1-989bfb8e-cce3-463d-9343-cc71d16796b1',
  'osUpdatedAt': '2022-06-30T19:23:43.925Z',
  'osCreatedAt': '2022-06-30T19:23:43.355Z',
  'osUpdatedBy': '469583b8-a486-4427-b383-0af1c4eecb86',
  'mobile': '50693540130',
  'osCreatedBy': '',
  'name': 'test name 50693540130',
  'email': '50693540130@example.com'},
 'name': 'Harshil',
 'osCreatedBy': '',
 'experience': [{'osUpdatedAt': '2022-06-30T19:23:43.925Z',
   'osUpdatedBy': '469583b8-a486-4427-b383-0af1c4eecb86',
   'institute': 'Some other school',
   'osid': '1-29ec18c1-a464-466b-a31e-5c96924f77ba'}],
 'identityDetails': {'osid': '1-01f845e4-19f4-47a8-a46d-d007ca8b0838',
  'osUpda

## invite a student

In [ ]:
userId

'50693540130'

In [ ]:
headers = {
    'content-type': 'application/json',
}

data = {"identityDetails":{
        "fullName":"Test %s"%userId, 
        "gender":"Male"
        },
        "educationDetails":[{
            "institute":"Some other school"
        }
        ]
       }
#{"name": "Prashant Joshi",\n\t\t"phoneNumber": "9876543210",\n\t\t"email": "prashant@upps.in",\n\t\t"school": "UP Public School"\n\t}


response = requests.post('http://localhost:8081/api/v1/Student/invite', headers=headers, json=data)


In [ ]:
response.json()



{'id': 'sunbird-rc.registry.invite',
 'ver': '1.0',
 'ets': 1656617024104,
 'params': {'resmsgid': '',
  'msgid': '9943d1dd-4c32-48b7-b34b-006d14eb7537',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK',
 'result': {'Student': {'osid': '1-f1d98d76-00df-4959-8b4a-52dc34f0954a'}}}

In [ ]:
sid = response.json()["result"]["Student"]["osid"]


In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%("Student", sid), headers=headers)
print(response.status_code)
response.json()


200


{'educationDetails': [{'institute': 'Some other school',
   'osid': '1-559fc1fd-4728-4529-b48e-5e6c669a40fd'}],
 'osid': '1-f1d98d76-00df-4959-8b4a-52dc34f0954a',
 'identityDetails': {'osid': '1-4c9105c4-0323-408a-8e42-9248f3eb4241',
  'gender': 'Male',
  'fullName': 'Test 50693540130'}}

In [ ]:
sid

'1-f1d98d76-00df-4959-8b4a-52dc34f0954a'

## claim

In [ ]:

headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

params = (
    ('send', 'true'),
)

data = {
    "entityName":"Student",
    "entityId":"%s"%sid,
    "name":"studentInstituteAttest",
    "additionalInput":{"idx": "%s"%sid}
}
response = requests.put('http://localhost:8081/api/v1/send', headers=headers, params=params, json=data)
print(response.status_code)
response.json()



200


{'id': 'sunbird-rc.registry.send',
 'ver': '1.0',
 'ets': 1656617024368,
 'params': {'resmsgid': '',
  'msgid': 'f5bffa69-4030-4b7f-813f-500140b9edfd',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK',
 'result': {'attestationOSID': '1-f7971adb-aff2-445f-aa01-179bc8620d64'}}

In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%("Student", sid), headers=headers)
print(response.status_code)
response.json()

200


{'educationDetails': [{'institute': 'Some other school',
   'osid': '1-559fc1fd-4728-4529-b48e-5e6c669a40fd'}],
 'studentInstituteAttest': [{'additionalInput': {'osid': '1-4bbaa122-272e-43fa-8e3a-137295941b7b',
    'idx': '1-f1d98d76-00df-4959-8b4a-52dc34f0954a'},
   'entityName': 'Student',
   '_osState': 'DRAFT',
   'name': 'studentInstituteAttest',
   'entityId': '1-f1d98d76-00df-4959-8b4a-52dc34f0954a',
   'osid': '1-f7971adb-aff2-445f-aa01-179bc8620d64',
   'propertyData': '{"name":"Test 50693540130","educationDetails":[{"institute":"Some other school","osid":"1-559fc1fd-4728-4529-b48e-5e6c669a40fd"}]}'}],
 'osid': '1-f1d98d76-00df-4959-8b4a-52dc34f0954a',
 'identityDetails': {'osid': '1-4c9105c4-0323-408a-8e42-9248f3eb4241',
  'gender': 'Male',
  'fullName': 'Test 50693540130'}}

## As a Teacher get claims

In [ ]:
token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJjQVJSY3JFdWZ5TnNUR1laZlBmNUZyVGtFVzgwVHpMaC11aTc2MEZCM3o4In0.eyJleHAiOjE2NTY2MTc2MjMsImlhdCI6MTY1NjYxNzAyMywianRpIjoiNTBhZWI5YzItYjFmZi00ZDViLTllM2MtOTVkZmRlMTZhZmQzIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI0Njk1ODNiOC1hNDg2LTQ0MjctYjM4My0wYWYxYzRlZWNiODYiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI2MjMxYjY0Zi00NjI3LTQxMjItYTUyNS0yNThlZDk5ZmNlMmUiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl1

In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/Teacher/claims', headers=headers)
print(response)
print(response.status_code, response.json())
claim_id = response.json()[-1]["id"]
response.json()

<Response [200]>
200 []


IndexError: list index out of range

## approve the claim

In [ ]:
claim_id

'96852340-a171-43b6-b357-d8666aafac60'

In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}
data = {
    "action":"GRANT_CLAIM",
}
response = requests.post('http://localhost:8081/api/v1/Teacher/claims/%s/attest'%claim_id, headers=headers, json=data)
print(response.status_code)
response.json()

200


{'resmsgid': '',
 'msgid': '4abdf275-4259-4d89-a40c-661e85bdabb8',
 'err': '',
 'status': 'SUCCESSFUL',
 'errmsg': ''}

In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/Teacher/claims', headers=headers)
print(response.status_code)
claim_id = response.json()[0]["id"]
response.json()

200


[{'id': '7e912173-329a-490f-9561-c9943ce6f693',
  'entity': 'Student',
  'entityId': '1-383ecafc-eea0-4bcc-a061-42336753ae81',
  'propertyURI': '',
  'createdAt': '2022-02-15T12:54:51.731+00:00',
  'attestedOn': '2022-02-15T12:57:40.834+00:00',
  'status': 'CLOSED',
  'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('Some other school'))",
  'attestorEntity': 'Teacher',
  'requestorName': None,
  'propertyData': '{"name":"Test 81540432738","institute":"Some other school"}',
  'attestationId': '1-524c50f7-7241-4c7d-966c-a4dc1089c94f',
  'attestationName': 'studentInstituteAttest',
  'closed': True},
 {'id': '325d7a14-900e-45d8-a928-82c92461969c',
  'entity': 'Student',
  'entityId': '1-8333e64a-4f4e-4d98-95b3-c4753f47acba',
  'propertyURI': '',
  'createdAt': '2022-02-15T07:57:08.877+00:00',
  'attestedOn': None,
  'status': 'OPEN',
  'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('Some other school'))",
  'attestorEntity': 'Teacher',
  'requestorName': None,

In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%("Student", sid), headers=headers)
print(response.status_code)
response.json()

200


{'educationDetails': [{'institute': 'Some other school',
   'osid': '1-45cecd72-ded5-4540-b3c8-861628537143'}],
 'studentInstituteAttest': [{'additionalInput': {'osid': '1-f8f46a25-8628-4e7e-94fa-2737018ea046',
    'idx': '1-f487ba74-2b5b-4d4b-9619-471e75aec617'},
   'entityName': 'Student',
   '_osState': 'PUBLISHED',
   'name': 'studentInstituteAttest',
   'entityId': '1-f487ba74-2b5b-4d4b-9619-471e75aec617',
   '_osAttestedData': '{"name":"Test 97767099632","institute":"Some other school"}',
   'osid': '1-12c772ad-6dcc-412b-b61c-318ded5a0961',
   '_osClaimId': '96852340-a171-43b6-b357-d8666aafac60',
   'propertyData': '{"name":"Test 97767099632","institute":"Some other school"}'}],
 'osid': '1-f487ba74-2b5b-4d4b-9619-471e75aec617',
 'identityDetails': {'osid': '1-ea45774c-cd00-4bab-8c82-f0bf361c4990',
  'gender': 'Male',
  'fullName': 'Test 97767099632'}}